In [ ]:
import pandas as pd
import numpy as np
from time import time
from IPython.display import display 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import fbeta_score, accuracy_score
from sklearn.preprocessing import StandardScaler

data = pd.read_csv("Training_Set.csv")

data.drop('DELIVERY_ID', axis=1, inplace = True)
data.drop('DELIVERY_DATE', axis=1, inplace = True)
data.drop('CUSTOMER_ID', axis=1, inplace = True)
'''array_ = data['CUSTOMER_ID'].unique()
d = dict(enumerate(array_))
res = dict((v,k) for k,v in d.items())
data['CUSTOMER_ID'] = data['CUSTOMER_ID'].map(res)'''


scaler = StandardScaler()

In [ ]:
data.head(3)

,CUSTOMER_ID,DELIVERY_VALUE_ORIG,DELIVERY_VALUE_MODIFIER,DELIVERY_VALUE,UNIQUE_ITEM_FRACTION,ITEM_PRICE_MEAN,CATEGORY_BOUNCE_RISK,PAY_UP_FRONT_DELIVERY_COUNT,PAY_AFTER_DELIVERY_COUNT,CANCELLED_ORDER_COUNT,DELIVERY_BOUNCED_PAYMENT_COUNT,CUSTOMER_DELIVERY_COUNT,ADDRESS_COUNT,OPEN_DEBT,PERCENTAGE_PRIVATE_LABEL,PERCENTAGE_A_BRAND,PERCENTAGE_ENTRY_LEVEL,FC_AGE,FRAUDULENT
0,0,35.71,4.9,40.61,0.8421,1.8795,0.0155,0,52,2,0,0,0,60.15,0.3750,0.6250,0.0000,234,0
1,1,105.75,-4.5,101.25,0.6102,1.7924,0.0202,0,81,21,0,0,3,79.67,0.1111,0.3611,0.5278,105,0
2,2,35.89,-4.1,31.79,0.3810,1.7090,0.0132,0,16,2,7,0,0,152.14,0.0000,0.3750,0.6250,105,0


In [ ]:
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split



data_scaled = scaler.fit_transform(data)

testSet = pd.read_csv("Test_Set.csv")
testSet.drop('DELIVERY_ID', axis=1, inplace = True)
testSet.drop('DELIVERY_DATE', axis=1, inplace = True)
testSet.drop('CUSTOMER_ID', axis=1, inplace = True)

X = data.drop('FRAUDULENT', axis=1)
y = data['FRAUDULENT']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1, random_state = 1)

print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 240675 samples.
Testing set has 26742 samples.


In [ ]:
len(testSet)

66851

In [ ]:
def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    results = {}
    
    # TODO: Fit the learner to the training data using slicing with 'sample_size' using .fit(training_features[:], training_labels[:])
    start = time() # Get start time
    learner = learner.fit(X_train[:sample_size], y_train[:sample_size])
    end = time() # Get end time
    
    # TODO: Calculate the training time
    results['train_time'] = end - start
        
    # TODO: Get the predictions on the test set(X_test),
    #       then get predictions on the first 300 training samples(X_train) using .predict()
    start = time() # Get start time
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train[:300])
    end = time() # Get end times
    
    # TODO: Calculate the total prediction time
    results['pred_time'] = end - start
            
    # TODO: Compute accuracy on the first 300 training samples which is y_train[:300]
    results['acc_train'] = accuracy_score(y_train[:300], predictions_train)
        
    # TODO: Compute accuracy on test set using accuracy_score()
    results['acc_test'] = accuracy_score(y_test, predictions_test)
    
    # TODO: Compute F-score on the the first 300 training samples using fbeta_score()
    results['f_train'] = fbeta_score(y_train[:300], predictions_train[:300], beta = 0.5)
        
    results['f_test'] = fbeta_score(y_test, predictions_test, beta = 0.5)
       
    # Success
    print("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))
        
    return results

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
clf_A = GradientBoostingClassifier(random_state=10)
clf_B = GaussianNB()
clf_C = RandomForestClassifier(random_state=12)

samples_100 = len(y_train)
samples_10 = int(samples_100 * 0.1)
samples_1 = int(samples_100 * 0.01)

results = {}
for clf in [clf_A, clf_B, clf_C]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([samples_1, samples_10]):
        results[clf_name][i] = \
        train_predict(clf, samples, X_train, y_train, X_test, y_test)

# Run metrics visualization for the three supervised learning models chosen
#vs.evaluate(results, accuracy, fscore)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


GradientBoostingClassifier trained on 2406 samples.


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


GradientBoostingClassifier trained on 24067 samples.
GaussianNB trained on 2406 samples.
GaussianNB trained on 24067 samples.


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


RandomForestClassifier trained on 2406 samples.
RandomForestClassifier trained on 24067 samples.


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [ ]:
results

{'GradientBoostingClassifier': {0: {'train_time': 1.0049171447753906,
   'pred_time': 0.04695463180541992,
   'acc_train': 1.0,
   'acc_test': 0.98994091690973,
   'f_train': 0.0,
   'f_test': 0.27305825242718446},
  1: {'train_time': 8.131479501724243,
   'pred_time': 0.04537343978881836,
   'acc_train': 1.0,
   'acc_test': 0.9927080996185775,
   'f_train': 0.0,
   'f_test': 0.45454545454545453}},
 'GaussianNB': {0: {'train_time': 0.006383180618286133,
   'pred_time': 0.015161991119384766,
   'acc_train': 0.9566666666666667,
   'acc_test': 0.9472365567272455,
   'f_train': 0.0,
   'f_test': 0.0744047619047619},
  1: {'train_time': 0.01556086540222168,
   'pred_time': 0.014460086822509766,
   'acc_train': 0.9633333333333334,
   'acc_test': 0.9608107097449704,
   'f_train': 0.0,
   'f_test': 0.1027077497665733}},
 'RandomForestClassifier': {0: {'train_time': 0.4549386501312256,
   'pred_time': 0.28034114837646484,
   'acc_train': 1.0,
   'acc_test': 0.9924089447311345,
   'f_train': 0.0

In [ ]:
testSet = pd.read_csv("Test_Set.csv")
testSet.drop('DELIVERY_ID', axis=1, inplace = True)
testSet.drop('DELIVERY_DATE', axis=1, inplace = True)
testSet.drop('CUSTOMER_ID', axis=1, inplace = True)



In [ ]:
output = clf_C.predict(testSet)
print(list(output))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
import json
import csv
finalTest = pd.read_csv("Test_Set.csv")
outputData = pd.DataFrame(output)
finalTest['LABEL'] = outputData

In [ ]:
finalTest['LABEL']

0        0
1        0
2        0
3        0
4        0
        ..
66846    0
66847    0
66848    0
66849    0
66850    0
Name: LABEL, Length: 66851, dtype: int64

In [ ]:
finalTest.to_csv("Group6Submit.csv")

In [ ]:

#20
324248c60e91

abe042200382